# National Health and Nutrition Examination Survey (NHANES)
[https://wwwn.cdc.gov/nchs/nhanes/](https://wwwn.cdc.gov/nchs/nhanes/)

Este laboratório usará dados extraídos do NHANES.

## NHANES datasets from 2013-2014

Foram extraídos dados do [NHANES 2013-2014](https://wwwn.cdc.gov/nchs/nhanes/continuousnhanes/default.aspx?BeginYear=2013).

Os dados estão disponíveis em arquivos CSV no diretório `/data/nhanes`. Lá está disponível a documentação do esquema dos arquivos.

# Tarefa 1 - Esquema Relacional

Analise os arquivos CSV que estão no diretório `/data/nhanes` e escreva um esquema relacional referente às tabelas, suas chaves primárias e estrangeiras (não é SQL ainda). Coloque a chave primária com dois underlines antes e dois depois:

\__teste\__

- Pessoa(__id__, gender, age)
- Medicamento(__code__, name)
- Uso(__person_id__, __drug_code__, days_use) 
    - __person_id__ é uma chave estrangeira para Pessoa.__id__
    - __drug_code__ é chave estrangeira para Medicamento.__code__

## Criação das Tabelas em SQL e Importação de Dados

Considere o seguinte exemplo em que é criada uma tabela em SQL e importado dados do arquivo CSV `demographic-person.csv`.

In [6]:
%defaultDatasource jdbc:h2:mem:db

In [18]:
DROP TABLE IF EXISTS Pessoa;

CREATE TABLE Pessoa (
  Id VARCHAR(5) NOT NULL,
  Gender INTEGER,
  Age INTEGER,
  PRIMARY KEY(Id)
) AS SELECT
    id,
    gender,
    age
FROM CSVREAD('../data/demographic-person.csv');

In [19]:
SELECT * FROM Pessoa;

# Tarefa 2 - Criação das Tabelas em SQL e Importação de Dados

Escreva instruções SQL para montar as duas tabelas restante, conforme definido no esquema relacional da **Tarefa 1**.

**Importante:** não defina chave primária para a tabela de uso de medicamento.

Durante a criação das tabelas, realize a importação dos dados do arquivo CSV nas tabelas criadas, conforme exemplo anterior.

In [20]:
DROP TABLE IF EXISTS Medicamento;

CREATE TABLE Medicamento (
  Code VARCHAR(6) NOT NULL,
  Name VARCHAR(100),
  PRIMARY KEY(Code)
) AS SELECT
    code,
    name
FROM CSVREAD('../data/medications-drug.csv');

SELECT * FROM Medicamento;

In [ ]:
DROP TABLE IF EXISTS Uso;

CREATE TABLE Uso (
    Person_Id VARCHAR(5) NOT NULL,
    Drug_Code VARCHAR(6) NOT NULL,
    Days INTEGER,
    
    FOREIGN KEY(Person_Id)
    REFERENCES Pessoa(Id)
        ON DELETE NO ACTION
        ON UPDATE NO ACTION,
    FOREIGN KEY(Drug_Code)
    REFERENCES Medicamento(Code)
        ON DELETE NO ACTION
        ON UPDATE NO ACTION
) AS SELECT
    person_id,
    drug_code,
    days_use
FROM CSVREAD('../data/medications-use.csv');

SELECT * FROM Uso;

# Tarefa 3 - Consultas Simples

Execute as consultas a seguir.

a) Liste pessoas com idade acima de 60

In [24]:
SELECT * FROM Pessoa WHERE Pessoa.Age > 60

b) Liste as idades das pessoas cadastradas (sem repetições)

In [25]:
SELECT DISTINCT Pessoa.Age FROM Pessoa 

c) Liste código das pessoas, idade, código dos medicamentos que usaram e dias de uso

In [26]:
SELECT Pessoa.Id, Pessoa.Age, Uso.Drug_Code, Uso.Days FROM Pessoa, Uso WHERE Pessoa.Id = Uso.Person_Id

d) Liste código das Pessoas, idade, nome dos medicamentos que usaram e quantos dias de uso

In [32]:
SELECT Pessoa.Id, Pessoa.Age, Medicamento.Name, Uso.Days FROM Pessoa, Uso, Medicamento WHERE Pessoa.Id = Uso.Person_Id AND Uso.Drug_Code = Medicamento.Code;

e) Liste nome dos medicamentos que foram usados por pessoas (sem repetições)

In [34]:
SELECT DISTINCT Medicamento.Name FROM Medicamento WHERE Medicamento.Code = ANY(SELECT Uso.Drug_Code FROM Uso)

f) Liste nome dos medicamentos usados por mais de 2.000 dias (sem repetições)

In [35]:
SELECT DISTINCT Medicamento.Name FROM Medicamento, Uso WHERE Medicamento.Code = Uso.Drug_Code AND Uso.Days > 2000